<a href="https://colab.research.google.com/github/sebaspector/Dogs-of-the-Dow/blob/main/DogsOfTheDow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip3 install --upgrade yfinance (or in Colab !pip install yfinance)
import pandas as pd
import yfinance as yf
import datetime

#DJI 30 companies
from google.colab import drive
drive.mount('/content/gdrive')
DJI = pd.read_csv('/content/gdrive/MyDrive/DOW 30 list.csv',sep=";")
tickers = DJI['Symbol'].to_list()
start = datetime.datetime(2020, 12, 31)
end = datetime.datetime(2021, 11, 17)

#Data from Yahoo
data = yf.download(tickers, start = start, end = end)
data.head()
data['Adj Close'].head()
prices = data['Adj Close'].copy()
pricesT = prices.T.copy()
pricesT.columns = pricesT.columns.astype('str').str[:10]
pricesT['last_price'] = pricesT['2021-11-16']


#Create data frame por stocks and sum of year 2021 dividends
data = pd.DataFrame()
for i in tickers:
    series = yf.Ticker(i).dividends.loc[start:end]
    data = pd.concat([data, series], axis=1)

data = data.fillna(0)
data.columns = tickers
sum_dividends = data.sum(axis=0)
sum_dividends.sort_index(inplace=True)

stock_price = pricesT['last_price']
div_yield = (sum_dividends / stock_price) * 100


#Dogs of the Dow
dogs_of_the_dow = div_yield.sort_values(ascending=False)[:10]
print(dogs_of_the_dow)

#Small Dogs of the Dow
stock_price.loc[dogs_of_the_dow.index] #Price of the Dogs of the Dow
small_dogs_of_the_dow = stock_price.loc[dogs_of_the_dow.index].sort_values(ascending=True)[:5]
print(small_dogs_of_the_dow)

#Performance stocks
pricesT['Performance'] = (pricesT['2021-11-16'] - pricesT['2021-01-04'])*100.0 / pricesT['2021-01-04']

#Performance Dogs of the Dow
table1 = pd.concat([dogs_of_the_dow,pricesT['Performance'] ], axis=1).reindex(dogs_of_the_dow.index)
df_stocks_dogs_performance = table1.drop(columns = 0)
weights = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
performance_dogs_of_the_dow = (df_stocks_dogs_performance['Performance'] * weights).sum(axis=0)
print("Performance for Dogs of the Dow: ", performance_dogs_of_the_dow)

#Performance Small Dogs of the Dow
table2 = pd.concat([small_dogs_of_the_dow,pricesT['Performance'] ], axis=1).reindex(small_dogs_of_the_dow.index)
df_stocks_small_dogs_performance = table2.drop(columns = 'last_price')
weights = [0.2,0.2,0.2,0.2,0.2]
performance_small_dogs_of_the_dow = (df_stocks_small_dogs_performance['Performance'] * weights).sum(axis=0)
print("Performance for Small Dogs: ", performance_small_dogs_of_the_dow)